In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir = '../data/PoliInfo2-EntityLinking-FormalRun-v20200703/AnswerSheet'
os.listdir(data_dir)

['PoliInfo2-EntityLinking-JA-Formal-Training.tsv',
 'PoliInfo2-EntityLinking-JA-Formal-Test.tsv']

In [3]:
df_train = pd.read_table(os.path.join(data_dir,'PoliInfo2-EntityLinking-JA-Formal-Training.tsv'))
df_test = pd.read_table(os.path.join(data_dir,'PoliInfo2-EntityLinking-JA-Formal-Test.tsv'))


In [4]:
df_train['形態素'] = df_train['形態素'].replace('　',' ')
df_test['形態素'] = df_test['形態素'].replace('　',' ').replace('\u3000\u3000\u3000━━━━━━━━━━━━━','━━━━━━━━━━━━━')


In [5]:
split_train_text = df_train['形態素'].values[1:]
split_test_text = df_test['形態素'].values[1:]


In [6]:
train_text = ''.join([w if w is str else str(w) for w in split_train_text])
test_text = ''.join([w if w is str else str(w) for w in split_test_text])


In [8]:
train_mention_names = list(df_train[df_train['IOB2']=='B']['メンション'].value_counts().keys())
print('train mention name: ', len(train_mention_names), '種類')
df_train[df_train['IOB2']=='B']['メンション'].value_counts()

train mention name:  101 種類


円滑化法                                            18
働き方改革を推進するための関係法律の整備に関する法律案                     14
ＩＲ法案                                            11
特定複合観光施設区域の整備の推進に関する法律案                         10
ＩＲ推進法案                                           9
経営承継円滑化法                                         9
推進法案                                             8
カジノ法案                                            7
労働安全衛生法の一部を改正する法律案                               6
環太平洋パートナーシップ協定の締結に伴う関係法律の整備に関する法律の一部を改正する法律案     6
一般職の職員の給与に関する法律等の一部を改正する法律案                      6
政治資金規正法                                          6
クリーンウッド法                                         5
グリーン購入法                                          5
推進法                                              5
特別職の職員の給与に関する法律の一部を改正する法律案                       5
金融円滑化法                                           5
戦争法                                              5
貸金業法                                             4
労働基準法                          

In [87]:
train_entity_names = list(df_train[df_train['IOB2']=='B']['wikipediaタイトル'].value_counts().keys())
print('train mention name: ', len(train_entity_names), '種類')
df_train[df_train['IOB2']=='B']['wikipediaタイトル'].value_counts()

train mention name:  46 種類


NIL                                        67
特定複合観光施設区域の整備の推進に関する法律                     60
中小企業者等に対する金融の円滑化を図るための臨時措置に関する法律           26
働き方改革関連法                                   19
中小企業における経営の承継の円滑化に関する法律                    11
平和安全法制                                      9
政治資金規正法                                     6
労働基準法                                       5
国等による環境物品等の調達の推進等に関する法律                     5
風俗営業等の規制及び業務の適正化等に関する法律                     5
貸金業法                                        4
財政法                                         4
総合保養地域整備法                                   3
工場三法                                        3
中小企業信用保険法                                   2
労働契約法                                       2
特定秘密の保護に関する法律                               2
関税暫定措置法                                     2
犯罪による収益の移転防止に関する法律                          2
所得税法                                        2
介護保険法                                       1
政党助成法                             

## Mention Detection using Regular Expression

In [120]:
import regex
import collections

In [102]:
p = regex.compile(r'[\p{Script=Han}\p{Script=Katakana}・a-zA-Z1-9]+')
print(p.findall('私はカジノ・賭博解禁法案と賭博案に賛成します。'))


['私', 'カジノ・賭博解禁法案', '賭博案', '賛成']


In [105]:
p = regex.compile(r'[\p{Script=Han}\p{Script=Katakana}・a-zA-Z1-9]+')
candidate_mentions = p.findall('私はカジノ・賭博解禁法案と賭博案に賛成します。')
candidate_mentions

['私', 'カジノ・賭博解禁法案', '賭博案', '賛成']

In [108]:
candidate_mentions = [m if m.endswith('法案') for m in candidate_mentions]


SyntaxError: invalid syntax (<ipython-input-108-785a55d37c9a>, line 1)

In [128]:
candidate_mentions = []
for m in p.finditer(train_text):
    if m.group().endswith(('法案','法制','法')) and not m.group().endswith('療法') and m.group() not in ['法','法案','法制']:
        candidate_mentions.append(m.group())
        
c = collections.Counter(candidate_mentions)

In [129]:
c.most_common()

[('実施法', 48),
 ('本法案', 38),
 ('刑法', 33),
 ('実施法案', 25),
 ('議員立法', 17),
 ('推進法案', 17),
 ('プログラム法', 15),
 ('特別法', 12),
 ('基本法', 11),
 ('方法', 11),
 ('円滑化法', 11),
 ('違法', 8),
 ('経営承継円滑化法', 8),
 ('立法', 7),
 ('カジノ法案', 7),
 ('推進法', 6),
 ('手法', 6),
 ('合法', 6),
 ('労働安全衛生法', 6),
 ('労働法制', 6),
 ('政治資金規正法', 5),
 ('民法', 5),
 ('金融円滑化法', 5),
 ('ンウッド法', 5),
 ('同法案', 4),
 ('戦争法', 4),
 ('中小企業金融円滑化法', 4),
 ('政府提出法案', 4),
 ('ン購入法', 4),
 ('ト法', 3),
 ('貸金業法', 3),
 ('今後実施法', 3),
 ('プログラム法案', 3),
 ('本法', 3),
 ('憲法', 3),
 ('小規模企業振興基本法', 3),
 ('整備法', 3),
 ('方改革関連法案', 3),
 ('国内法', 2),
 ('司法', 2),
 ('カジノ解禁法案', 2),
 ('風営法', 2),
 ('閣法', 2),
 ('選定方法', 2),
 ('地方交付税法', 2),
 ('特定秘密保護法', 2),
 ('平和安全法制', 2),
 ('中小企業信用保険法', 2),
 ('機構法', 2),
 ('独法', 2),
 ('関税暫定措置法', 2),
 ('関連法案', 2),
 ('労働基準法', 2),
 ('過労死促進法', 2),
 ('対処法', 1),
 ('犯収法', 1),
 ('犯罪収益移転防止法', 1),
 ('カジノ法', 1),
 ('規制法案', 1),
 ('後実施法', 1),
 ('現行刑法', 1),
 ('基本法案', 1),
 ('実行法', 1),
 ('カジノ解禁法', 1),
 ('特別立法', 1),
 ('一番法', 1),
 ('競馬法', 1),
 ('競輪法', 1),
 ('風適法', 1),
 (